In [ ]:
import plotly.plotly as py
import plotly.figure_factory as ff

import numpy as np
import pandas as pd

import bokeh
from bokeh.io import show
from bokeh.models import LogColorMapper, CustomJS, Toggle, ColumnDataSource, CDSView, GroupFilter
from bokeh.palettes import Viridis6 as palette
from bokeh.plotting import figure
from bokeh.layouts import column, row, widgetbox 

from bokeh.models.widgets import Slider
from bokeh.layouts import Column

from bokeh.sampledata.us_counties import data as counties

In [ ]:
df = pd.read_csv("clean.csv")

In [ ]:
def county_name_format(name):
    final_name = ""
    try:
        if type(name) != 'float':
            words = name.split(" ")
            if words[-1] == 'COUNTY':
                words.pop()
            for i in range(0, len(words)):
                final_name += words[i].capitalize() + " "
            final_name = final_name.rstrip()
        else:
            final_name = np.nan
    except:
        pass
    return(final_name)

In [ ]:
counties_ca = {
    code: county for code, county in counties.items() if (county["state"] == "ca")
}

In [ ]:
counties_ca

In [ ]:
county_xs = [county["lons"] for county in counties_ca.values()]
county_ys = [county["lats"] for county in counties_ca.values()]
county_names = [county['name'] for county in counties_ca.values()]

In [ ]:
def replace_nan(county_info):
    for l in county_info:
        for idx, item in enumerate(l):
            if np.isnan(item):
                l[idx] = np.nanmean(l)
    return(county_info)

In [ ]:
b = [[1,2,np.nan], [3,2,1], [np.nan, 1, 5]]

In [ ]:
replace_nan([[1,2,np.nan], [3,2,1], [np.nan, 1, 5]])

In [ ]:
county_xs = replace_nan(county_xs)
county_ys = replace_nan(county_ys)

In [ ]:
len(county_names)

In [ ]:
df['recipient_location_county_name'] = df['recipient_location_county_name'].apply(lambda name: county_name_format(name))

In [ ]:
sum_obligation = df.groupby(['fiscal_year','recipient_location_county_name'])['total_obligation'].median().reset_index()

In [ ]:
sum_obligation = sum_obligation[sum_obligation['recipient_location_county_name']!= '']

In [ ]:
years = sum_obligation['fiscal_year'].unique()

In [ ]:
for year in years:
    sub = sum_obligation[sum_obligation['fiscal_year']==year]
    temp = pd.DataFrame(columns=['fiscal_year', 'recipient_location_county_name', 'total_obligation'])
    for county in county_names:
        if county not in sub['recipient_location_county_name'].unique():
            temp = temp.append({'fiscal_year': year, 'recipient_location_county_name': county, 'total_obligation': 0}, ignore_index=True)
    for county in sub['recipient_location_county_name'].unique():
        if county not in county_names:
            sum_obligation = sum_obligation[sum_obligation['recipient_location_county_name']!= county]
    sum_obligation = sum_obligation.append(temp)
    

In [ ]:
sum_obligation = sum_obligation.sort_values(by=['fiscal_year', 'recipient_location_county_name']).reset_index()

In [ ]:
sum_obligation

In [ ]:
for year in years:
    sub = sum_obligation[sum_obligation['fiscal_year']==year]
    print(sub.shape)

In [ ]:
# data=dict(
#     x=county_xs,
#     y=county_ys,
#     name=county_names,
#     rate=sample['total_obligation'],
# )

In [ ]:
years = sum_obligation['fiscal_year'].unique()
data_list = []
for year in years:
    data = dict(
        x=county_xs,
        y=county_ys,
        name=county_names,
        year=str(year),
        rate=sum_obligation[sum_obligation['fiscal_year']==year]['total_obligation']
    )
    data_list.append(data)
plot_df = pd.DataFrame(data_list[0])
for i in range(1, len(data_list)):
    try:
        temp = pd.DataFrame(data_list[i])
        plot_df = plot_df.append(temp)
    except:
        print(i)

In [ ]:
plot_df

In [ ]:
data = ColumnDataSource(plot_df)
view = CDSView(source = data, filters = [GroupFilter(column_name = "year", group = "2001")])

In [ ]:
callback = CustomJS(args = {"source": data, "view": view}, code = """
    console.log("Hey dummy")
    view.filters[0].group = cb_obj.value.toString();
    source.change.emit();
""")

slider = Slider(start=2001, end=2019, value=2001, title="Year")

slider.js_on_change("value", callback)

In [ ]:
# # add slider with callback to update data source
# slider = Slider(start=2001, end=2019, value=2001, step=1, title="Year")

In [ ]:
TOOLS = "pan,wheel_zoom,reset,hover,save"

p = figure(
    title="CA spending", tools=TOOLS,
    x_axis_location=None, y_axis_location=None,
    tooltips=[
        ("Name", "@name"), ("Spending)", "@rate"), ("(Long, Lat)", "($x, $y)")
    ])
p.grid.grid_line_color = None
p.hover.point_policy = "follow_mouse"

p.patches('x', 'y', source=data, view = view,
          fill_color={'field': 'rate', 'transform': LogColorMapper(palette=palette)},
          fill_alpha=0.7, line_color="black", line_width=0.5)

layout = Column(slider, p)
show(layout)

In [ ]:
exp = plot_df.copy()

In [ ]:
len(exp['rate'])

In [ ]:
import random
exp['rate'] = random.sample(range(20, 4000), 1102)

In [ ]:
data = ColumnDataSource(exp)
view = CDSView(source = data, filters = [GroupFilter(column_name = "year", group = "2001")])

In [ ]:
np.nanmean([1,2,np.nan])